# "Bookmakers do not want to limit the amount of money that are being bet by bettors. They want to balance the earnings with the givings."

###                                                         We want to undertand how bookmakers
#### 1) calculate the odds between two teams (suppose there is no tie)
#### 2) change the odds accordingly to the amount that is being played among those two choices
#### 3) change the odds based on real-time occurancies (e.g. injuries/red cards, new coach/player, form etc.)


### Below i've written a code that calculates the odds for the two teams to win accordingly to the bets that have been placed.


In [92]:
import random


class DynamicBookmaker:
    def __init__(self, initial_odds=(2.0, 2.0), margin=0.05):
        self.margin = margin
        self.bets = {'TeamA': 0, 'TeamB': 0}
        self.odds = list(initial_odds)

    def calculate_odds(self):
        total_money = sum(self.bets.values())
        if self.bets['TeamB'] == 0 or self.bets['TeamA']==0 :
            return 0
            
        else:
            odds_team_a = (total_money / (self.bets['TeamA'] )) * (1 - self.margin)
            odds_team_b = (total_money / (self.bets['TeamB'] )) * (1 - self.margin)
            return odds_team_a, odds_team_b
    def take_bet(self, team, amount):
        if team in self.bets:
            self.bets[team] += amount
            self.update_odds()

    def settle_bets(self, winner):
        if winner in self.bets:
            if winner == 'TeamA':
                team_w = 0
            else:
                team_w = 1
            total_money_won = self.bets[winner] * self.odds[team_w]
            
            self.update_odds()
            return total_money_won
        else:
            print("Invalid winner.")
            return 0

    def update_odds(self):
        self.odds = self.calculate_odds()


# Example usage:
bookmaker = DynamicBookmaker()

# Simulation: Taking bets
bookmaker.take_bet('TeamA', 100)
bookmaker.take_bet('TeamB', 100)

# Simulation: Retrieving and printing updated odds
odds_team_a, odds_team_b = bookmaker.odds
print(f"Team A Odds: {odds_team_a:.2f}, Team B Odds: {odds_team_b:.2f}")

# Simulation: Settling bets
winner = random.choice(['TeamA', 'TeamB'])
print(f"Winner: {winner}")
total_money_won = bookmaker.settle_bets(winner)

print(f"Total money that were bet: {sum(bookmaker.bets.values())}")
print(f"Total money won by players: {total_money_won}")
print(f"Commission ({bookmaker.margin}%) for bookmakers: {sum(bookmaker.bets.values())-total_money_won}")




Team A Odds: 1.90, Team B Odds: 1.90
Winner: TeamB
Total money that were bet: 200
Total money won by players: 190.0
Commission (0.05%) for bookmakers: 10.0


#### Now, think that some experts give the initial odds, but still the odds are calculated by the amount of  money that is being bet.


In [87]:
import random
import sys

class DynamicBookmakerV2:
    def __init__(self, initial_odds, margin):
        self.margin = margin
        self.bets = {'TeamA': 0, 'TeamB': 0}
        self.odds = list(initial_odds)
        print(f'Initial odds TeamA: {initial_odds[0]}, TeamB = {initial_odds[1]}')

    def calculate_odds(self):
        total_money = sum(self.bets.values())
        if self.bets['TeamB'] == 0 or self.bets['TeamA']==0 :
            return 0
            
        else:
            odds_team_a = (total_money / (self.bets['TeamA'] )) * (1 - self.margin)
            odds_team_b = (total_money / (self.bets['TeamB'] )) * (1 - self.margin)
            if odds_team_a < 1.01:
                odds_team_a = 1.01
            if odds_team_b < 1.01:
                odds_team_b = 1.01
            if odds_team_a > 11.13:
                odds_team_a = 11.13
            if odds_team_b > 11.13:
                odds_team_b = 11.13
                
            return odds_team_a, odds_team_b

    def take_bet(self, team, amount):
        if team in self.bets:
            if amount > 5000:
                
                sys.exit("Max bet is 5000")   
            else:
                self.bets[team] += amount
                self.update_odds()

    def settle_bets(self, winner):
        if winner in self.bets:
            if winner == 'TeamA':
                team_w = 0
            else:
                team_w = 1
            total_money_won = self.bets[winner] * self.odds[team_w]
            
            self.update_odds()
            return total_money_won
        else:
            print("Invalid winner.")
            return 0

    def update_odds(self):
        self.odds = self.calculate_odds()


# Example usage:
bookmaker = DynamicBookmakerV2((2,2),0.07)

# Simulation: Taking bets
bookmaker.take_bet('TeamA', 456)
bookmaker.take_bet('TeamB', 5000)

# Simulation: Retrieving and printing updated odds
odds_team_a, odds_team_b = bookmaker.odds
print(f"Team A Odds: {odds_team_a:.2f}, Team B Odds: {odds_team_b:.2f}")

# Simulation: Settling bets
winner = random.choice(['TeamA', 'TeamB'])
print(f"Winner: {winner}")
total_money_won = bookmaker.settle_bets(winner)

print(f"Total money that were bet: {sum(bookmaker.bets.values())}")
print(f"Total money won by players: {total_money_won}")
print(f"Commission ({bookmaker.margin}%) for bookmakers: {sum(bookmaker.bets.values())-total_money_won}")




Initial odds TeamA: 2, TeamB = 2
Team A Odds: 11.13, Team B Odds: 1.01
Winner: TeamA
Total money that were bet: 5456
Total money won by players: 5074.08
Commission (0.07%) for bookmakers: 381.9200000000001


#### We now want to eliminate the excessive and rapid change in the odds due to a big amount of money that has been bet on a side. Stategies that cope with this problem are implying a max bet, human intervation, setting limits, algorithmic safeguards.

In [88]:
1/2.27 + 1/1.65



1.0465892404218395

### However we can understand that the initial bets can't be 0, if noone has bet yet

In [144]:
import random

class BookmakerSimulation:
    def __init__(self, initial_odds=(2.0, 2.0), margin=0.05):
        self.margin = margin
        self.odds = list(initial_odds)
        self.total_money = {'TeamA': 0, 'TeamB': 0}

    def calculate_implied_probabilities(self):
        total_money = sum(self.total_money.values())
        implied_prob_team_a = (1 / self.odds[0]) * (1 + self.margin)
        implied_prob_team_b = (1 / self.odds[1]) * (1 + self.margin)
        return implied_prob_team_a, implied_prob_team_b

    def adjust_odds(self):
        implied_prob_team_a, implied_prob_team_b = self.calculate_implied_probabilities()

        # Simulating a basic adjustment algorithm (you can replace this with a more sophisticated one)
        adjustment_factor = random.uniform(0.9, 1.1)
        adjusted_prob_team_a = implied_prob_team_a * adjustment_factor
        adjusted_prob_team_b = implied_prob_team_b * (1 / adjustment_factor)

        # Recalculate odds based on adjusted probabilities
        self.odds[0] = 1 / adjusted_prob_team_a
        self.odds[1] = 1 / adjusted_prob_team_b

    def take_bet(self, team, amount):
        if team in self.total_money:
            self.total_money[team] += amount
            self.adjust_odds()

    def display_odds(self):
        print(f"Current Odds - Team A: {self.odds[0]:.2f}, Team B: {self.odds[1]:.2f}")

# Example usage:
bookmaker = BookmakerSimulation()

# Simulating bets
bookmaker.take_bet('TeamA', 10000)
bookmaker.take_bet('TeamB', 8000)

# Displaying initial odds
bookmaker.display_odds()

# Simulating more bets and dynamic odds adjustment
for _ in range(5):
    team = random.choice(['TeamA', 'TeamB'])
    amount = random.randint(1000, 5000)
    bookmaker.take_bet(team, amount)

    # Displaying updated odds after each bet
    bookmaker.display_odds()


Current Odds - Team A: 2.14, Team B: 1.54
Current Odds - Team A: 1.94, Team B: 1.54
Current Odds - Team A: 1.79, Team B: 1.51
Current Odds - Team A: 1.59, Team B: 1.55
Current Odds - Team A: 1.67, Team B: 1.33
Current Odds - Team A: 1.76, Team B: 1.15
